In [ ]:
from autumn.core.runs import ManagedRun

In [ ]:
region = "northern_territory"

In [ ]:
run_id = "sm_sir/northern_territory/1661081125/2f04228"

In [ ]:
mr = ManagedRun(run_id)

In [ ]:
pbi = mr.powerbi.get_db()